In [1]:
from collections import Counter

from preprocessing import preprocess
import plotly.express as px
import pyfaidx 
import pandas as pd
from pathlib import Path
from Bio import SeqRecord, SeqIO


In [2]:
files = ('../data/IIPR002547.genes.fasta', '../random.fasta', "../IPR045863.faa")

datasets = {str(filename) : list(SeqIO.parse(filename, format="fasta")) for filename in Path("../data/raw").glob("*.fasta")}
datasets = {filename: preprocess(list(genes)) for filename, genes in datasets.items()}


datasets

{'../data/raw/IPR045863.fasta': [SeqRecord(seq=Seq('ATGACGTGTTTCCCTCCACGGGAAGAAAGACAAGGGGACAACTCGTTTCAGCAA...TAG'), id='lcl|JAPDRO010000002.1_cds_KAJ9631096.1_1', name='lcl|JAPDRO010000002.1_cds_KAJ9631096.1_1', description='lcl|JAPDRO010000002.1_cds_KAJ9631096.1_1 [locus_tag=H2203_001627] [db_xref=InterPro:IPR002523,InterPro:IPR045863,PFAM:PF01544] [protein=hypothetical protein] [protein_id=KAJ9631096.1] [location=join(1999907..2000143,2000197..2000523,2000573..2000704,2000754..2000942,2001005..2001172)] [gbkey=CDS]', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGACCACCAATGCCGTCTGGTACGGCTTTGATATGAAGGAACATTATAAGTAT...TAG'), id='lcl|JAOQBA010000019.1_cds_KAJ4007675.1_1', name='lcl|JAOQBA010000019.1_cds_KAJ4007675.1_1', description='lcl|JAOQBA010000019.1_cds_KAJ4007675.1_1 [locus_tag=NW752_010344] [db_xref=InterPro:IPR045863] [protein=hypothetical protein] [protein_id=KAJ4007675.1] [location=join(501164..501892,501939..502091,502140..502328)] [gbkey=CDS]', dbxrefs=[]),
  SeqRecord(seq=Seq('ATGACC

In [3]:
df = pd.DataFrame.from_records([(filename, len(gene.seq)) for filename, genes in datasets.items() for gene in genes], columns=['filename', 'gene_length'])

px.violin(df, color='filename', x='gene_length')

In [15]:
records = []
for filename, genes in datasets.items():
    all_nucs = ''.join(str(gene.seq) for gene in genes)
    counts = Counter(all_nucs)
    for nuc, count in counts.items():
        records.append((filename, nuc, count / len(all_nucs)))


df = pd.DataFrame.from_records(records, columns=["filename", "nucleotide", "Relative Frequency"])

px.bar(df, barmode="group", x="nucleotide", y="Relative Frequency", color="filename")

In [16]:
def kmerize(sequence: str, k=3):
    return [sequence[i:i+k] for i in range(len(sequence) - k)]

records = []
for filename, genes in datasets.items():
    all_kmers  = [kmer for gene in genes for kmer in kmerize(str(gene.seq))]
    counts = Counter(all_kmers)
    for kmer, count in counts.items():
        records.append((filename, kmer, count / len(all_kmers)))


df = pd.DataFrame.from_records(records, columns=["filename", "kmer", "Relative Frequency"])

px.violin(df, violinmode="group", hover_name="kmer", y="Relative Frequency", color="filename", points="all")

In [6]:
import ssw

In [20]:
SeqIO.write(datasets["../data/raw/IPR045863.fasta"], "../data/preprocessed/IPR045863.fasta", format='fasta')

665

In [7]:
help(ssw.Alignment)

Help on class Alignment in module ssw.alignmenttuple:

class Alignment(builtins.tuple)
 |  Alignment(CIGAR: str, optimal_score: int, sub_optimal_score: int, reference_start: int, reference_end: int, read_start: int, read_end: int, cigar_pair_list: List[Tuple[int, str]])
 |  
 |  Alignment namedtuple structure.  Used for storing results of
 |  :meth:`ssw.alignmentmgr.AlignmentMgr.align`
 |  
 |  Attributes:
 |      CIGAR: The CIGAR string
 |      optimal_score: The optimal alignment score (SSW primary score)
 |      sub_optimal_score: The sub-optimal alignment score (SSW secondary score)
 |      reference_start: The reference start index
 |      reference_end: The reference end index
 |      read_start: The read start index
 |      read_end: The read end index
 |      cigar_pair_list: Convenience list of cigar length code pairs
 |  
 |  Method resolution order:
 |      Alignment
 |      builtins.tuple
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getnewargs__(self)
 

In [8]:
help(ssw.force_align)

Help on cython_function_or_method in module ssw.alignmentmgr:

force_align(read: 'STR_T', reference: 'STR_T', force_overhang: 'bool' = False, aligner: 'Optional[AlignmentMgr]' = None) -> 'Alignment'
    Enforces no gaps by raising the ``gap_open`` penalty
    
    Args:
        read: Read sequence python string or bytes-string
        reference: Reference sequence python string or bytes-string
        force_overhang: Make sure only one end overhangs
        aligner: pass an existing :class:`AlignmentMgr` object
    
    Returns:
        :class:`ssw.alignmenttuple.Alignment` result
    
    Raises:
        ValueError: No solution found
        ValueError: Read does not align to one overhang



In [9]:
mgr = ssw.AlignmentMgr()
help(mgr.build_dna_score_matrix())


Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [ ]:
len(genes[0])

1317

In [ ]:
dir(pyfaidx.Fasta("../data/raw/IIPR002547.genes.fasta"))

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close',
 'faidx',
 'filename',
 'get_seq',
 'get_spliced_seq',
 'items',
 'keys',
 'mutable',
 'records',
 'values']

In [24]:
aligned = list(SeqIO.parse("../data/aligned/IPR045863.fasta", format="fasta"))

In [29]:
import numpy as np

matrix = np.array([np.array(list(gene.seq)) for gene in aligned])

In [30]:
matrix.shape

(665, 10008)

In [35]:
matrix[:, 10:20]

array([['-', '-', '-', ..., '-', '-', '-'],
       ['-', '-', '-', ..., '-', '-', '-'],
       ['-', '-', '-', ..., '-', '-', '-'],
       ...,
       ['-', '-', '-', ..., 'C', 'G', 'A'],
       ['-', '-', '-', ..., 'C', 'G', 'A'],
       ['-', '-', '-', ..., 'C', 'G', 'A']], dtype='<U1')

In [36]:
from collections import Counter

Counter(len(set(column)) for column in matrix.T)

Counter({2: 4568, 5: 3054, 3: 1569, 4: 817})